# Зависимость времени работы от параметров тензора

In [2]:
%load_ext autoreload


In [3]:

%autoreload 2

import time
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import numpy as np
import pandas as pd

from bilinear_package.src import hadamard_product, random_tensor_generation, rounding, primitives

from experiments_stuff import get_tensor_for_experiment, default_dict_to_df, count_time

## Во всех экспериментов параметры по умолчанию: число ядер - 10, размер моды - 20, размер изначального ранга = ранг округления - 8, количество повторений - 1

In [4]:
default_config = {
    "kernels" : 10,
    "modes" : 20,
    "initial_rank" : 8,
    "rounding_rank" : 8,
    "tries" : 1
}

In [5]:
list(zip(np.arange(20, 80, 5), np.ones(13, dtype = int) * 10))

[(20, 10),
 (25, 10),
 (30, 10),
 (35, 10),
 (40, 10),
 (45, 10),
 (50, 10),
 (55, 10),
 (60, 10),
 (65, 10),
 (70, 10),
 (75, 10)]

In [7]:
from bilinear_package.src.hadamard_product import approximateHadamardProduct


count_time(default_config, ["kernels", "tries"], list(zip(np.arange(10, 201, 10), np.ones(20, dtype = int) * 15)), approximateHadamardProduct, "experiments_results/experiment1/kernels.txt")

,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200
,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15
0,0.020939,0.028394,0.039598,0.059737,0.072245,0.082654,0.117989,0.170775,0.178152,0.197361,0.220121,0.242459,0.265775,0.284836,0.307923,0.317050,0.334811,0.379261,0.366848,0.406241
1,0.020299,0.027386,0.040730,0.055480,0.088460,0.082536,0.123186,0.167463,0.176415,0.202188,0.216221,0.238169,0.256464,0.283485,0.300800,0.315971,0.334669,0.374252,0.373776,0.404589
2,0.017469,0.027931,0.039956,0.054963,0.070247,0.082268,0.139826,0.167532,0.181655,0.201445,0.216679,0.228352,0.254992,0.282199,0.302052,0.314995,0.343584,0.374975,0.367987,0.402300
3,0.017491,0.028663,0.040941,0.054392,0.067970,0.082383,0.153040,0.165517,0.183766,0.202306,0.213840,0.240294,0.262674,0.284487,0.307923,0.314593,0.356396,0.383473,0.369074,0.397832
4,0.015678,0.027278,0.040042,0.054916,0.079632,0.082228,0.159374,0.167756,0.183061,0.201044,0.215802,0.240720,0.266191,0.277775,0.295644,0.315545,0.347883,0.348611,0.369778,0.405647
5,0.015092,0.028370,0.041577,0.057912,0.079536,0.082392,0.155934,0.167607,0.180157,0.202268,0.217338,0.234371,0.265253,0.277856,0.307305,0.325406,0.345983,0.380140,0.374689,0.395257
6,0.015262,0.027860,0.042558,0.055029,0.071830,0.082272,0.154373,0.167480,0.183920,0.202485,0.215502,0.232452,0.265120,0.285550,0.291236,0.324024,0.347202,0.379392,0.367633,0.401543
7,0.014385,0.027678,0.047443,0.055871,0.071830,0.083050,0.154595,0.166628,0.182625,0.202460,0.209622,0.237960,0.257488,0.285951,0.297990,0.323642,0.344494,0.372932,0.380484,0.406922
8,0.014774,0.027554,0.051517,0.054942,0.069500,0.081229,0.146655,0.164712,0.176821,0.198661,0.211509,0.235349,0.265788,0.275858,0.306522,0.324572,0.347394,0.367468,0.377570,0.393043


In [24]:
count_time(default_config, ["modes", "tries"], list(zip(np.arange(10, 120, 10), np.ones(15, dtype = int) * 20)), approximateHadamardProduct, "experiments_results/experiment1/modes.txt")

,10,20,30,40,50,60,70,80,90,100,110
,20,20,20,20,20,20,20,20,20,20,20
0,0.011827,0.013271,0.016865,0.022384,0.027879,0.033731,0.041453,0.047569,0.052619,0.059100,0.066676
1,0.008693,0.013765,0.016883,0.022895,0.028141,0.034650,0.043893,0.048884,0.052339,0.057786,0.064805
2,0.009045,0.014000,0.016652,0.022690,0.031174,0.033776,0.040548,0.047266,0.052381,0.058660,0.065566
3,0.008667,0.014499,0.016859,0.024102,0.027717,0.033397,0.043149,0.049056,0.052849,0.057822,0.065377
4,0.010203,0.014459,0.016871,0.023149,0.028204,0.034148,0.040109,0.046999,0.052634,0.062679,0.065882
5,0.008247,0.013906,0.018450,0.023311,0.028653,0.033237,0.040672,0.049206,0.052355,0.063197,0.064616
6,0.008344,0.016362,0.018903,0.023294,0.029866,0.033703,0.042969,0.047249,0.055061,0.058699,0.066737
7,0.007839,0.014688,0.018293,0.024852,0.028188,0.034119,0.040535,0.047315,0.056603,0.058506,0.064645
8,0.007805,0.015425,0.017690,0.022939,0.028085,0.033892,0.044742,0.047514,0.053448,0.058114,0.065220


In [28]:
count_time(default_config, ["initial_rank", "rounding_rank", "tries"], list(zip(np.arange(5, 62, 4), np.arange(5, 62, 4), np.ones(100, dtype = int) * 10)), approximateHadamardProduct, "experiments_results/experiment1/ranks.txt")

,5,9,13,17,21,25,29,33,37,41,45,49,53,57,61
,5,9,13,17,21,25,29,33,37,41,45,49,53,57,61
,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10
0,0.007128,0.021020,0.074631,0.167902,0.436063,1.476409,2.369942,3.545681,5.065446,6.781760,9.183902,12.408658,17.699619,22.339018,22.553182
1,0.006503,0.019292,0.055837,0.154155,0.483998,1.475592,2.326375,3.464987,4.945129,6.695216,8.943904,12.723924,17.149952,23.765704,20.537968
2,0.006498,0.021342,0.056492,0.145355,0.452109,1.459500,2.322119,3.506501,4.890037,6.728488,9.104097,12.866998,17.939682,22.312610,21.641443
3,0.006421,0.019302,0.056596,0.146361,0.352401,1.518665,2.330706,3.490887,4.908847,6.770859,9.001523,12.749263,14.759783,19.439852,25.147197
4,0.006283,0.018628,0.101695,0.143934,0.351027,1.451450,2.350098,3.495305,4.915077,6.742844,9.224073,12.576635,15.260048,19.020846,24.397182
5,0.004324,0.018919,0.080264,0.145919,0.357919,1.449914,2.337443,3.470453,4.967293,6.942858,9.102062,12.861486,15.502519,18.204652,23.499417
6,0.004059,0.018666,0.092192,0.146118,0.376770,1.431221,2.343948,3.706761,4.926846,6.633112,9.208227,12.734328,16.354746,18.891484,23.346932
7,0.004580,0.020610,0.055185,0.147228,0.364685,1.414219,2.380099,3.880911,4.917524,6.736929,9.109031,12.944557,15.707379,15.878579,22.721402
